In [4]:
import pandas as pd
import os
from itertools import product, combinations

In [2]:
cwd = os.chdir('Engage3')

In [7]:
auditors = pd.read_csv('auditors.csv')
prices = pd.read_csv('prices.csv')
stores = pd.read_json('stores.json')

In [116]:
p_s_join = prices.merge(stores, on='Store ID')
full_join = p_s_join.merge(auditors, on='Auditor ID')

In [117]:
banner_x_region = p_s_join.pivot_table('Price', ['Banner', 'UPC'], 'Region')
banner_x_region.head(10)

Region            Kansas  New York  Northern California  Texas
Banner  UPC                                                   
Safeway 11873171     NaN       NaN                 6.09   5.19
        15052612   53.99       NaN                  NaN  54.49
        16482322   17.89       NaN                  NaN  18.09
        16729338    7.99       NaN                 9.39   8.09
        16829288    3.59       NaN                 4.19   3.59
        16900911   29.19       NaN                34.19  29.49
        16999755    5.29       NaN                 6.19   5.29
        17066659    3.09       NaN                 3.59   3.09
        19696884     NaN       NaN                  NaN   5.89
        19911643     NaN       NaN                  NaN  30.69

In [ ]:
#results export to CSV
banner_x_region.to_csv('output.csv')

In [73]:
#sanity check of NaN prices, to see whether store exists in region
pd.crosstab(stores.Region, stores.Banner)

Banner,Kroger,Safeway,Trader Joes,Walmart,Wegmans,Whole Foods
Region,,,,,,
Hawaii,1,1,1,1,1,1
Kansas,1,1,1,1,1,1
New York,1,0,1,1,1,1
Northern California,1,1,1,1,0,1
Texas,1,1,1,1,1,1


In [101]:
#check region counts for imbalanced data
banner_x_region.agg('count')

Region
Kansas                 3087
New York               3025
Northern California    1634
Texas                  3765
dtype: int64

In [127]:
#check count per auditor, workload definitely looks unequally distributed among auditors
full_join.groupby(['First','Last','Region_x']).size()

First  Last     Region_x           
Bob    Smith    Northern California     916
Dave   Johnson  Kansas                 1302
Jack   Smith    New York               1784
                Northern California     306
Jerry  Johnson  Texas                  2288
Jill   Smith    New York               1241
Mike   Johnson  Kansas                 1785
Randy  Johnson  Texas                  1477
Sue    Smith    Northern California     412
dtype: int64

In [155]:
#drop NaNs to examine price ratios on region and banner dimensions
shared_products = banner_x_region.dropna()

In [182]:
#drop and reload to get rid of multi-index for easier analysis
shared_products.to_csv('shared.csv')
shared_simple = pd.read_csv('shared.csv')

In [243]:
#testing whether price ratios hold on region and banner
combos = list(product(set(shared_simple.Banner),combinations(shared_simple.columns[2:],2)))

pd.DataFrame.from_records([[x, min(shared_simple[shared_simple['Banner']==x[0]][x[1][0]]/shared_simple[shared_simple['Banner']==x[0]][x[1][1]]), max(shared_simple[shared_simple['Banner']==x[0]][x[1][0]]/shared_simple[shared_simple['Banner']==x[0]][x[1][1]])] for x in combos]).rename(columns={0: 'combination tested', 1: 'min ratio', 2:'max ratio'})

,combination tested,min ratio,max ratio
0,"(Whole Foods, (Kansas, New York))",0.023140,2.884058
1,"(Whole Foods, (Kansas, Northern California))",0.020526,2.884058
2,"(Whole Foods, (Kansas, Texas))",0.023844,3.372881
3,"(Whole Foods, (New York, Northern California))",0.816514,1.154725
4,"(Whole Foods, (New York, Texas))",1.000000,1.342686
5,"(Whole Foods, (Northern California, Texas))",1.145138,1.224719
6,"(Walmart, (Kansas, New York))",0.921260,1.011765
7,"(Walmart, (Kansas, Northern California))",0.837736,0.914894
8,"(Walmart, (Kansas, Texas))",0.760458,1.061728
9,"(Walmart, (New York, Northern California))",0.875472,0.947761


In a nutshell, I'm trying to control for one of two dimensions: region or banner. When I take a given store and get the price ratio between any two states, it should be about the same all the way down the column, since only the region multiplier is in effect. Most of the combinations have a fairly close spread, however some Whole Foods records are way off. It's clear to see why when you look at the raw data - every Whole Foods price for Kansas is the same. Looks like someone forgot the where condition on their SQL update...

In [250]:
shared_simple[shared_simple['Banner']=='Whole Foods'].head()

,Banner,UPC,Kansas,New York,Northern California,Texas
176,Whole Foods,16999755,1.99,5.79,6.49,5.59
177,Whole Foods,17066659,1.99,3.39,3.79,3.29
178,Whole Foods,21872584,1.99,8.79,9.89,8.59
179,Whole Foods,22976162,1.99,54.69,61.59,53.09
180,Whole Foods,23493525,1.99,28.89,32.59,27.99
